# Robust Statistics

In [ ]:
from scipy.stats import multivariate_normal as mvn
import scipy.stats as stats
import pylab
import numpy as np
import pandas as pd
import statsmodels.api as sm
import mplfinance as fplt
import math
from matplotlib import pyplot as plt
import scipy
import BarSampler as bs
def get_log_return(df):
    log_returns = pd.Series(np.log((df['close'] - df['open'])/df['open'] + 1),name='log returns')
    return pd.concat([df,log_returns], axis=1)

### MCD

The MCD estimator (Rousseeuw, 1984) is an often used high-breakdown and affine equivariant estimator of location and scatter:
For fixed $h$, with $[n + p + 1]/2 􏰜\leq h \leq􏰜 n$

1. $\hat{\pmb{\mu}}_0$ is the mean of the $h$ observations for which the determinant of the sample covariance matrix is minimal.
2. $\hat{\pmb{\Sigma}}_0$ is the covariance matrix of that subset multiplied by a consistency factor

The MCD estimator can only be computed when $h > p$, otherwise the covariance matrix of any $h$-subset will be singular. This condition is certainly satisfied when $n \geq 􏰝 2p$. It is however recommended that $n \geq􏰝 5p$.

One approach to the minimum covariance determinant estimator was proposed by Rousseeuw P.J., and LeRoy A.M. (1987). A practical approach to the computation of this estimator is the Fast MCD algorithm developed in Rousseeuw and Van Driessen (1999). Let n be the size of the data set and p the number of variables in the data set. The goal of the Fast MCD algorithm is to find a subset of the data of size h who's covariance matrix has the minimum determinant. As shown in Lopuhaa H.P., and Rousseeuw P.J. (1991) the finite breakdown value of the MCD estimator is $(n-h+1)/n$ where h is chosen so that $[n+p+1]/2\leq h\leq n$. This allows the user to choose $h$ in order to establish a tradeoff between the amount of data included in the computation and the level of robustness desired. Below we outline the Fast MCD algorithm and provide code for it's implementation. The steps of Fast MCD are as follows.

1. The default h is $[n+p+1]/2$, but the user may choose any integer $h$ such that $[n+p+1]/2 \leq h \leq n$. The program reports the breakdown value $(n-h+1)/n$. If you are sure that the data set contains less than 25% contamination a good compromise between breakdown value and statistical efficiency is $h=[0.75n]$.

2. if $h=n$ the the MCD is just the usual sample mean and covariance matrix. Report these and terminate.

3. if $p=1$ (univariate data), compute the MCD as follows.<br>
	3a. Sort the data into a list and partition the list into subset ${k,\ldots,k+h-1}$ for $k=1,\ldots,n-h+1$<br>
	3b. Compute the sample mean and variance of each subset.<br>
	3c. Return the mean and variance of the subset from 3b which had the smallest variance.

4. Suppose that $h<n$ and $p>=2$. If $n$ is small (say, $n \leq 600$) then repeat the following (say) 500 times.<br>
	4a. Construct a subset $H_1$ as follows. Draw a random $p+1$ length subset $J$ and compute $T_0 = Mean(J)$ and $S_0=Cov[J]$. If $|S_0|=0$ then extend $J$ by adding another random observations until $|S_0|>0$.<br>Compute the distances $d_0^2(i) = (x_i - T_0)^T S_0^{-1}(x_i - T_0)$ and sort into $d_0(\pi(1)\leq \cdots \leq d_0(\pi(n))$ for $H_1 = \{\pi(1),\ldots,\pi(n)\}$.<br>
	4b. (The C-step) Given a subset $H_{old}$ with the estimates $T_{old}$ and $S_{old}$ preform the following<br>
Compute the distances $d_{old}(i)$ for $i=1,\ldots,n$.<br>Sort these distances which yields a permutation $\pi$ for which$d_{old}(\pi(1)\leq \cdots \leq d_{old}(\pi(h))$.<br>Put $H_{new}=\{\pi(1),\ldots,\pi(h)\}$.<br>Compute $T_{new}=Mean(H_{new})$ and $S_{new}=Cov(H_{new})$.<br>
	4c. For the 10 results in part 4b with the lowest determinant carry out the C-step until $|S_{new}|=0$ or $|S_{old}|=|S_{new}|$.<br>
	4d. Report the solution $(T,S)$ with the lowest determinant.

5. If n is large (say, n>600) then<br>
	5a. Construct 5 disjoint subsets $n_{sub}$ of size$[n/5]$ as follows.<br> Sample from the data set without replacement. The first $[n/5]$ samples go in the first subset, and so on.<br>
	5b. Inside each subset repeat 100 times.<br>Construct an initial subset $H_1$ of size $h_{sub}=[n_{sub}(h/n)]$<br>Carry out two C-steps using $n_{sub}$ and $h_{sub}$.<br>keep the best ten results $T_{sub}$ and $S_{sub}$.<br>
	5c. Pool the subsets yielding a merged set and for each of the 50 solutions<br>Carry out two C-steps using $n_{merged}$ and $h_{merged}$.<br>For the 10 results with the lowest determinant carry out the C-step until $|S_{new}|=0$ or $|S_{old}|=|S_{new}|$.<br>Report the solution $(T,S)$ with the lowest determinant.

The MCD estimate of the covariance matrix has two issues: it is neither unbiased or consistent. Pison G., Van Aelst S., and Wilems G. (2002) show that consistency factors can be computed to make the estimator consistant for the normal model. However it should be noted that for small data sets the factors do not make the estimator unbiased. Suppose that $\Sigma_{MCD}$ is the MCD estimate of our covariance matrix. It is shown in Pison G., Van Aelst S., and Wilems G. (2002) that by rescaling out estimate by $$I_{\alpha}\Sigma_{MCD}$$

with $I_{\alpha} = \frac{\alpha}{F_{\chi^2_{p+2}}(q_{\alpha})}$ and $q_{\alpha}=\chi^2_{p,\alpha}$ for $0.5\leq\alpha\leq 1$. This scaling makes MCD a consistant estimator for the normal model as shown in Croux C. and Gentiane H. (1998). 

The MCD is a robust estimator of scatter, but it does come at a cost. Some of the positive points are that
1. The influence function is bounded
2. The size of the subset $h$ determines the breakdown value
3. $\epsilon^*_n = min\left(\frac{n-h+1}{n},\frac{h-p}{n}\right)$
4. The maximum breakdown values occurs at $h = \frac{[n+p+1]}{2}$

Some standard choices are $\alpha = \frac{h}{n} = 0.5$ and $\alpha = 0.75$. These result in breakdown values of 50% and 25% respectively. 

The problem with MCD is the while it is asymptotically normal, it has low efficiency. For example, with $\alpha = 0.5$, the asymptotic relative efficiency of the diagonal elements of the MCD scatter matrix with respect to the sample covariance matrix, at the normal model, is only 6% when $p = 2$, and 20.5% when $p = 10$. With $\alpha = 0.75$ the relative efficiencies are 26.2% for $p = 2$ and 45.9% for $p = 10$

The efficiency of the MCD can be increased by applying a reweighting step.

First compute $RD_i = \sqrt{(\pmb{x}_i-\hat{\pmb{\mu}}_0)^T \hat{\pmb{\Sigma}}_0^{-1}(\pmb{x}_i-\hat{\pmb{\mu}}_0)}$

Next we compute a set of weights
$$w_i = \left(\begin{array}{lcr}
1 & if & RD_i \leq \sqrt{\chi^2_{p,0.975}}\\
0 & & otherwise
\end{array}\right.$$

Then the reweighted MCD estimates are given by
$$\hat{\mu}_{RMCD} = \frac{\sum_{i=1}^n w_i x_i}{\sum_{i=1}^n w_i}$$
$$\hat{\Sigma}_{RMCD} = \frac{1}{\sum_{i=1}^n w_i -1}\sum_{i=1}^n w_i(x_i-\hat{\mu}_{RMCD})(x_i-\hat{\mu}_{RMCD})^T$$

This has the advantage of not changing the breakdown value while increasing the efficiency. When $\alpha = 0.5$ we have 45.5% efficiency at $p=2$ and 82% at $p=10$

## DetMCD

Deterministic algorithm for MCD (Hubert et al., 2012).

Overall idea:<br>
Compute several ’promising’ h-subsets, based on
1. transformations of variables
2. easy-to-compute robust estimators of location and scatter. Apply C-steps until convergence.<br>
Apply C-steps until convergence.

This yields a fast algorithm which is at least as robust as FAST-MCD, but not fully affine equivariant.

In order to impliment this we need to do some preprocessing of the data. Start by standardizing X by subtracting the columnwise median and dividing by the columnwise $Q_n$ scale estimator. This 
1. Makes the final estimates location and scale equivariant.
2. Yields the standardized dataset $Z$ with rows $z_i$ and columns $Z_j$ .

We now apply the algorithm:<br>
First compute six initial estimate for $\hat{\mu}_k(Z)$ and $\hat{\Sigma}_k(Z)$ for location and scatter by:
1. Obtain six preliminary estimates $S_k$ for covariance/correlation matrix of $Z$.
2. Compute eigenvectors $E$ of $S_k$ and put $B = ZE$.
3. Estimate covariance of $Z$ by $\hat{\Sigma}_k(Z) = ELE^T$ with $L=diag(Q_n(B_1)^2,\ldots,Q_n(B_n)^2$.
4. Estimate the location with $\hat{\mu}_k(Z) = \hat{\Sigma}_k(Z)^{1/2}(med(Z\hat{\Sigma}_k(Z)^{1/2}))$
 

For each initial estimate do:
1. Compute the statistical distances $d_{ik} = d(z_i,\hat{\mu}_k(Z),\hat{\Sigma}_k(Z))$.
2. Let the inital subset $h_0$ be $h_0 = [\frac{n}{2}]$ oberservations with smallest $d_{ik}$.
3. Compute the statistical distances $d_{ik}$ based on these $h_0$ observations. Take the $h$ points with smallest $d_{ik}$ and apply C-steps until convergence.

Retain the $h$-subset with smallest covariance determinant.

This algorithm has some important advantages over FAST-MCD:
1. In moderate dimensions (say, $p=10$): faster than FAST-MCD and equally robust
2. In higher dimensions: faster than FAST-MCD and more robust, especially when there is much contamination
3. Deterministic: does not depend on any random selection 
4. Permutation invariant
5. Nearly affine equivariant
6. Initial estimates do not yet depend on the value $h$ which determines the breakdown value. This makes it easy to compute DetMCD for several h-values, and to see whether at some $h$ there is a substantial change in the objective function or the estimates (“monitoring”).

This leaves the obvious question when should we use FAST-MCD vs DetMCD? Here are some rules of thumb
1. When $p \leq 10$ run FAST-MCD.
2. When p is larger than this it becomes harder or even infeasible to draw enough initial subsets, and then it is better to run DetMCD.<br>


DetMCD is useful as a building block for multivariate analysis (multivariate regression, exponential smoothing, calibration, . . .)

## MVE

the MVE estimator (Rousseeuw, 1985), or minimum volume ellipsoid, is one of the oldest robust covariance estimators that is affine equivariant and has a positive breakdown value.

For fixed $h$ with $\frac{n+p+1}{2} \leq h \leq n$
$$(\hat{\mu},\hat{\Sigma}) = argmin_{\mu, \Sigma}|\hat{\Sigma}|$$
over all real $\mu$ and symmetric positive definite $\Sigma$ such that
$$\sqrt{(\pmb{x}_i-\hat{\pmb{\mu}})^T \hat{\pmb{\Sigma}}^{-1}(\pmb{x}_i-\hat{\pmb{\mu}})}\leq c^2$$

And the ellipsoid covers at least $h$ data point and has $p+1$ data points on it's boundary.

A generalization of this idea leads to the notion of an **S-estimator** (Rousseeuw and Leroy 1987). They are defined as follows
$$(\hat{\mu},\hat{\Sigma}) = argmin_{\mu, \Sigma}|\hat{\Sigma}|$$
over all real $\mu$ and symmetric positive definite $\Sigma$ such that
$$\frac{1}{n}\sum_{i=1}^n \rho(d_i)=\delta$$
with $d_i = \sqrt{(\pmb{x}_i-\hat{\pmb{\mu}})^T \hat{\pmb{\Sigma}}^{-1}(\pmb{x}_i-\hat{\pmb{\mu}})}$ and $\rho$ a smooth bounded function.

We can use $\delta$ to obtain efficiency for the normal model by setting it to
$$\delta = \mathbb{E}_{N_p(0,I)}[\rho(||X||)]$$

S-estimators are asymptotically normal. Their efficiency at the gaussian model is somewhat better than the efficiency of the RMCD, especially in higher dimensions.

For example, the diagonal element of the bisquare S scatter matrix with 50% breakdown value has an asymptotic relative efficiency of 50.2% for $p=2$,and 92% for $p=10$. (RMCD:45.5% for $p=2$ and 82% for $p=10$).

S-estimators are smoothed versions of the MVE, which corresponds to a function ρ that only takes on the values 0 and 1.

The breakdown value of both the location and scatter estimator is:
$$\epsilon^* = min \left(\frac{\delta}{\rho(\infty)},1-\frac{\delta}{\rho(\infty)}\right)$$

To compute an S-estimator, the FAST-S algorithm can be used (Salibian-Barrera and Yohai, 2006). It is similar to FAST-MCD.

## MM-Estimators

MM-estimators combine high robustness with high efficiency. They are based on two rho functions $\rho_0$ and $\rho_1$. The first $\rho$ function is chosen to obtain a high breakdown value. The second $\rho$ function is chosen to achieve a high efficiency.

To construct an MM-estimator, note that a scatter matrix can be separated into a scale estimate and a shape matrix by putting $\Gamma = |\Sigma|^{-1/p}\Sigma$, then
$$|\Gamma| = 1 \; and \; \Sigma = |\Sigma|^{1/p}\Gamma$$

We call $|\Sigma|^{1/2p}$ the **scale estimate** and $\Gamma$ the **shape estimate**.

Let $(\tilde{\mu},\tilde{\Sigma})$ be an S-estimator with rho function $\rho_0$. Denote 
$\sigma^2 =|\tilde{\Sigma}|^{1/p}$.

(Tatsuoka and Tyler, 2000) The MM-estimator for location and shape $(\hat{\mu},\hat{\Gamma})$ minimizes
$$\frac{1}{n}\sum_{i=1}^n\rho_1\left(\frac{\sqrt{(\pmb{x}_i-\hat{\pmb{\mu}})^T \hat{\pmb{\Gamma}}^{-1}(\pmb{x}_i-\hat{\pmb{\mu}})}}{\hat{\sigma}}\right)$$

over all real $\hat{\mu}$ and symmetric positive definite matrices with $|\Gamma|=1$. The MM-estimator for the covariance matrix is then $\hat{\Sigma} = \hat{\sigma}^2\hat{\Gamma}$

The location and shape estimates inherit the breakdown value of the auxiliary scale. Thus one typically chooses an S-estimator with 50% breakdown value.
For a bisquare $\rho_0$, $c = 1.547$ yields a 50% breakdown value.

The influence functions (and thus asymptotic variance) of MM-estimators for location and scatter equal those of M-estimators of location and scatter that use the function $\rho_1$.
For a bisquare $\rho_1$, $c = 4.685$ yields 95% efficiency (at the normal model).

However, MM-estimators with high efficiency are less robust.
In particular, they tend to give too much weight to ‘fairly nearby’ outliers, unlike methods with a ‘hard’ objective function like MCD and MVE.

## Lookahead and the Source-Target Concept

### Lookahead

Lookahead is the general term used to refer to a condition in a model which causes it to fail to be a filtration, http://en.wikipedia.org/wiki/Filtration_(mathematics). In the context of probability theory the model forms a nested series of $\sigma$-algebras. In simple terms it means that at time t a trading strategy only has access to data available up to but not including t. This insures that the model is causal, i.e.,  that it can actually be implemented in the real world. 

While this may seem an obvious requirement, lookahead is one of the most common (and dangerous) errors made in building realistic simulations of trading strategies. For example, there may be no overt lookahead in a trading strategy but some estimates, such as volatility used to normalize data, may be based on the entire dataset. Measurements for a specific time period are often available after some material amount of time has passed, so, e.g., quarterly estimates of earnings per share for the quarter ending June 31, 2012 are not available on July 01, 2012. Current databases available often do not contain information on companies or instruments that are no longer in existence and this survivor bias is a form of lookahead that can introduce serious survivor bias into a modeling effort.

In one sense lookahead is practically unavoidable. You only get on first look at the data. No matter how diligent you are in ensuring that your modeling efforts avoid lookahead, the fact that you have run different models over the same data means that some form of lookahead exists. This is even true if you use various cross-validation techniques such as having separate calibration and hold-out periods in your work.

### Sources and Targets

It is generally useful to develop and communicate in terms of sources and targets. If we are dealing with discrete times, ${0, 1, 2, \ldots, t-1, t, t+1, \ldots}$. The sources associated with time $t$ must only contain information from time $t-1$ and earlier. The targets associated with time $t$ must only contain information from $t$ and later. Building routines and data structures in which time is aligned in this fashion is useful and tends to reduce simple lookahead errors.

For example, given data $\pmb{x} = {x_0, \ldots, x_t, \ldots, x_{20}}$ and a problem in which the model was based on the prior 5 observations to forecast the next 3 periods, one could write a function that would return the source-target pair for the data $\pmb{x}$ and target time $t$. For example, for $t = 11$, the routine would select the following sub-vectors from $\pmb{x}$:

<img src="files/SourceTarget.png">

with the arrow indicating the t where the beginning of the target occurs. These pairs would be the basis for fitting the model, performing computations.

Using a routine to consistently perform this task reduces the chances for simple lookahead. Again, while it may seem excessive, unintentional lookahead is one of the most common modeling errors in finance. It is easy to do and tends to produce excellent performance which is, of course, completely unrealizable in practice.

## Exponential Smoothing

Exponential smoothing (http://en.wikipedia.org/wiki/Exponential_smoothing) is a smoothing and forecasting method based on weights that decay exponentially the further back in time observations are observed. If we are smoothing a process $x_{t-1}$ to produce a forecast $s_t$], we begin with a smoothing parameter $0 < \lambda < 1$ and initial condition $s_0$:
$$s_1 = \lambda x_0 + (1-\lambda)s_0$$
$$s_t = \lambda x_{t-1} + (1-\lambda)s_{t-1}$$
Thus, the forecast or smooth for the present time $t$ (i.e., $s_t$)  is a weighted average of the previous observation and the previous forecast. In most cases, the initial condition is assumed to be 0, but if there is a better estimate then there is no reason not to use it.

If we assume history trails off into infinity, then the initial condition goes away and the iteration above is equivalent to:
$$s_t = \lambda \sum_{i=1}^{t-1} (1-\lambda)^i x_{t-i} + (1-\lambda)^t s_0$$

This form motivates the name exponential smoothing. The forecast employs an exponentially damped set of weights applied to the history. Further, examining the weights themselves it is straightforward to show:
$$\lambda \sum_{i=1}^{t-1} (1-\lambda)^i = 1-(1-\lambda)^t \rightarrow \lambda \sum_{i=1}^{t} (1-\lambda)^i=1$$

The weights add to unity; hence, this estimate is also called an exponentially smoothed moving average. Exponential smoothing is extremely easy to apply. It only requires a single parameter and  a single value $s_t$ to bootstrap the forecast from one time step to the next; nevertheless, it accomplishes the relatively sophisticated and intuitively appealing task of exponentially down weighting the past to estimate the future.

The parameter $\lambda$ has a good interpretation as the model's half life $h$, i.e. the period at which the weight placed on an observation is half that of the initial weight. To see this take
$$(1-\lambda)^h = \frac{1}{2} \rightarrow \lambda = 1 - 2^{-1/h} \rightarrow h = -\frac{ln(2)}{ln(1-\lambda)}$$
Thus, a half-life of $h = 20$ periods (roughly a month for daily data) sets $\lambda \approx 0.034$, and a smoothing parameter $\lambda = 0.1$ sets the model's half life to $h \approx 6.6$ periods.

### Volatility Smoothing

Volatility smoothing is important. We want to use data across time and across a number of instruments, but the raw data displays large variations in volatility both cross-temporally and cross-sectionally.